In [37]:
%reset -f
#%matplotlib qt
import sys
import mne
import matplotlib.pyplot as plt
import numpy as np
import os.path as op
import os
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

#pick_ch=['P9','P7','P5','P3','PO7','PO3','O1','P10','P8','P6','P4','PO8','PO4','O2']
#%load_ext autoreload
#%autoreload 2
%run EEG_auxiliary_module.ipynb
%run general_tools.ipynb
mne.__version__
mne.set_log_level("WARNING")

infolder, outfolder = find_folder()
subs=['17']
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys

logging.basicConfig(filename = 'nn_log.log',format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO)

In [38]:
%run EEG_auxiliary_module.ipynb
data=list()
labels = list()
trials=4
for j in subs:
    fname = op.join(outfolder,'nn_img_S'+j+'-epo.fif')
    epochs = mne.read_epochs(fname)
    #trials = len(epochs['101']) # average all trials.
    blocks=len(epochs['31'])/trials
    #epochs = block_average(epochs, trials, blocks, kind='perc')
    X, Y = convert_epochs_to_3D_array(epochs)
    data.append(X)
    labels.append(Y)

In [39]:
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from braindecode.models.eegnet import EEGNetv4, Conv2dWithConstraint, EEGNet
from torch import nn
from braindecode.torch_ext.util import set_random_seeds

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = False
set_random_seeds(seed=20170629, cuda=cuda)
n_classes = 2
in_chans = data[0].shape[1]
# final_conv_length = auto ensures we only get a single output in the time dimension
model = EEGNetv4(in_chans=in_chans, n_classes=n_classes,
                        input_time_length=data[0].shape[2],
                        final_conv_length='auto')
if cuda:
    model.cuda()

In [40]:
from braindecode.torch_ext.optimizers import AdamW
import torch.nn.functional as F
#optimizer = AdamW(model.parameters(), lr=1*0.01, weight_decay=0.5*0.001) # these are good values for the deep model
optimizer = AdamW(model.parameters(), lr=0.0625 * 0.01, weight_decay=0)
model.compile(loss=F.nll_loss, optimizer=optimizer, iterator_seed=1,)

In [41]:
def comp_result(X,Y,tr_idx,tst_idx,vld_idx):
    from braindecode.datautil.signal_target import SignalAndTarget
    from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
    from torch import nn
    from braindecode.torch_ext.util import set_random_seeds
    from braindecode.torch_ext.optimizers import AdamW
    import torch.nn.functional as F

    train_set = SignalAndTarget(X[tr_idx], y=Y[tr_idx])
    valid_set = SignalAndTarget(X[vld_idx], y=Y[vld_idx])
    test_set = SignalAndTarget(X[tst_idx], y=Y[tst_idx])

    # Set if you want to use GPU
    # You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
    cuda = False
    set_random_seeds(seed=20170629, cuda=cuda)
    n_classes = 2
    in_chans = train_set.X.shape[1]
    # final_conv_length = auto ensures we only get a single output in the time dimension
    model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes,
                            input_time_length=train_set.X.shape[2],
                            final_conv_length='auto')
    if cuda:
        model.cuda()


    #optimizer = AdamW(model.parameters(), lr=1*0.01, weight_decay=0.5*0.001) # these are good values for the deep model
    optimizer = AdamW(model.parameters(), lr=0.0625 * 0.01, weight_decay=0)
    model.compile(loss=F.nll_loss, optimizer=optimizer, iterator_seed=1,)
    model.fit(train_set.X, train_set.y, epochs=30, batch_size=64, scheduler='cosine',
             validation_data=(valid_set.X, valid_set.y),)
    result = model.evaluate(test_set.X, test_set.y)
    return result

In [42]:
import itertools
import pickle
ms_all = []
results_all = []
for k in range(len(data)):
    combs = list(itertools.combinations(np.unique(labels[k]), 2))
    missclass = []
    results_sub =[]
    for idx,comb in enumerate(combs):
        X = data[k]*1e6
        Y = labels[k]
        ind_i = comb[0]
        ind_j = comb[1]
        X = X[(Y==ind_i) | (Y==ind_j),:,:].astype(np.float32)
        Y = Y[(Y==ind_i) | (Y==ind_j)]
        Y = np.equal(Y, np.ones(Y.shape[0])*ind_i)
        Y = Y.astype(np.int64)
        output_all = []
        misclass = []
        for i in range(10):
            np.random.seed(i)
            num = X.shape[0]
            new_vec = np.random.permutation(num)
            tr_idx = new_vec[0:46]
            vld_idx = new_vec[46:76]
            tst_idx = new_vec[76:]
            result = comp_result(X,Y,tr_idx,tst_idx,vld_idx)
            output_all.append(result)
            misclass.append(result['misclass'])
        missclass.append(misclass)
        results_sub.append(output_all)
    ms_all.append(missclass)
    results_all.append(results_sub)
with open('results.pkl', 'wb') as handle:
    pickle.dump(results_all, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('ms_all.pkl', 'wb') as handle:
    pickle.dump(ms_all, handle, protocol=pickle.HIGHEST_PROTOCOL)


NameError: name 'results' is not defined

In [43]:
a = np.array(ms_all)

In [46]:
np.mean(a)

0.483103448275862

In [2]:
import pickle

results = pickle.load( open( "results.pkl", "rb" ) )

In [12]:
len(data)


15

[[[{'loss': 1.0432415008544922,
    'misclass': 0.4051724137931034,
    'runtime': 0.0006206035614013672},
   {'loss': 1.2511610984802246,
    'misclass': 0.5517241379310345,
    'runtime': 0.0006167888641357422},
   {'loss': 1.2484385967254639,
    'misclass': 0.5086206896551724,
    'runtime': 0.0005342960357666016},
   {'loss': 1.1626602411270142,
    'misclass': 0.5258620689655172,
    'runtime': 0.0005171298980712891},
   {'loss': 1.0742833614349365,
    'misclass': 0.47413793103448276,
    'runtime': 0.0005908012390136719},
   {'loss': 0.9912470579147339,
    'misclass': 0.43965517241379315,
    'runtime': 0.0004992485046386719},
   {'loss': 1.016441822052002,
    'misclass': 0.43965517241379315,
    'runtime': 0.00051116943359375},
   {'loss': 1.0722707509994507,
    'misclass': 0.4913793103448276,
    'runtime': 0.0002930164337158203},
   {'loss': 0.939121425151825,
    'misclass': 0.48275862068965514,
    'runtime': 0.0005218982696533203},
   {'loss': 1.2655529975891113,
    '

In [ ]:
X.shape

In [ ]:
len(missclass)

In [ ]:
a = np.array(missclass)

In [ ]:
a.

In [ ]:
mnelab